In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt

In [2]:
dataset = 'kumar-4-hard'
data_path = "./dataset/{}-filtered/10X/".format(dataset)
labels_path = "./dataset/{}-filtered/labels.csv".format(dataset)
markers_path = "./results/aggregate/{}/markers.csv".format(dataset)

In [3]:
adata = sc.read_10x_mtx(
    data_path,
    var_names='gene_symbols',
    cache=False
)

In [4]:
y_df = pd.read_csv(labels_path, index_col=0)
y_df

,cluster.ids
cell,
Cell1,4
Cell10,2
Cell100,3
Cell101,2
Cell102,3
...,...
Cell95,3
Cell96,1
Cell97,3


In [5]:
y_df = pd.DataFrame(adata.obs_names, columns=["cell"]).join(y_df, on="cell")
y_df

,cell,cluster.ids
0,Cell1,4
1,Cell2,4
2,Cell3,4
3,Cell4,2
4,Cell5,1
...,...,...
494,Cell496,3
495,Cell497,3
496,Cell498,3
497,Cell499,4


In [6]:
mask = ~np.isnan(np.array(y_df['cluster.ids'])).reshape(-1)
mask[mask==False]

array([], dtype=bool)

In [7]:
y_df['cluster.ids'][mask]

0      4
1      4
2      4
3      2
4      1
      ..
494    3
495    3
496    3
497    4
498    1
Name: cluster.ids, Length: 499, dtype: int64

In [8]:
y = np.array(y_df['cluster.ids'][mask])

In [9]:
clusters, counts = np.unique(y, return_counts=True)
clusters, counts

(array([1, 2, 3, 4]), array([105,  61, 210, 123]))

In [10]:
weights = counts[np.argsort(clusters)]

In [11]:
def apply_classifier(X, y):
    clf = RandomForestClassifier()
    y_pred = cross_val_predict(clf, X, y, cv=5)
    f1 = f1_score(y, y_pred)
    return f1

markers_df = pd.read_csv(markers_path)
tools = markers_df.tool.unique()

f1_markers = {}
for tool in tools:
    f1_markers_tool = []
    for cluster in clusters:
        y_bin = np.array(y==cluster, dtype=int)
        markers = markers_df[
            (markers_df['cluster']==cluster) & (markers_df['tool']==tool)
           ].gene.unique()
        X_markers = adata[mask, markers].X.toarray()
        f1_markers_tool.append(apply_classifier(X_markers, y_bin))
    f1_markers[tool] = round((weights*np.array(f1_markers_tool)).sum()/weights.sum(), 3)

f1_all = []
for cluster in clusters:
    y_bin = np.array(y==cluster, dtype=int)
    X_all = adata[mask, ].X.toarray()
    f1_all.append(apply_classifier(X_all, y_bin))
f1_weighted = round((weights*np.array(f1_all)).sum()/weights.sum(), 3)

print("F1 weighted when training on markers")
print(f1_markers)
print("F1 weighted when training on all genes")
print(f1_weighted)

F1 weighted when training on markers
{'monocle': 0.992, 'scanpy': 0.989, 'seurat': 0.989, 'scvi': 0.989}
F1 weighted when training on all genes
0.919


In [12]:
import sys
from joblib import Parallel, delayed
def process(tool):
    step = 1
    tmp_scores = {}
    for cluster in clusters:
        n_markers = len(markers_df[markers_df['cluster']==cluster])   
        y_bin = np.array(y==cluster, dtype=int)
        for i in range(step, n_markers+step, step):
            markers = markers_df[
            (markers_df['cluster']==cluster) & (markers_df['tool']==tool) & (markers_df['rank']<=i)
        ].gene.unique()
            X = adata[mask, markers].X.toarray()
            f1 = apply_classifier(X, y_bin)
            if tool not in tmp_scores:
                tmp_scores[tool] = {}
            if cluster not in tmp_scores[tool]:
                tmp_scores[tool][cluster] = {}
            if 'scores' not in tmp_scores[tool][cluster]:
                tmp_scores[tool][cluster]['scores'] = []
            tmp_scores[tool][cluster]['scores'].append(f1)
        if 'mean' not in tmp_scores[tool][cluster]:
            tmp_scores[tool][cluster]['mean'] = []
        # mean for each cluster
        tmp_scores[tool][cluster]['mean'].append(np.mean(tmp_scores[tool][cluster]['scores']))
        print("Cluster {} done for tool {}".format(cluster, tool), file=sys.stderr)
        
    return tmp_scores

In [13]:
# -------- train with increasing # of features taken from markers rank --------
step = 1
tools = ['scvi', 'seurat', 'scanpy', 'monocle']

import pickle, os
if os.path.exists(f'scores_{dataset}.pickle'):
    with open(f'scores_{dataset}.pickle', 'rb') as handle:
        scores = pickle.load(handle)
else:
    scores = Parallel(n_jobs=len(tools))(delayed(process)(tool) for tool in tools)
    with open(f'scores_{dataset}.pickle', 'wb') as handle:
        pickle.dump(scores, handle, protocol=pickle.HIGHEST_PROTOCOL)   


# for i in range(step, n_markers+step, step):
#     sumForI = 0
#     for j, cluster in enumerate(clusters):
#         sumForI += (tmp_scores[cluster]['scores'][i-1] * counts[j])
#     if 'TotalMean' not in scores[tool]:
#         tmp_scores[tool]['TotalMean'] = []
#     tmp_scores['TotalMean'].append(sumForI/np.sum(counts))

# x_ticks = [i for i in range(step, n_markers+step, step)]
# # 4 axes plot
# fig, ax = plt.subplots(2, 2, figsize=(10, 10))
# plt.xticks(x_ticks, x_ticks)
# plt.plot(x_ticks, scores, marker='o')
# plt.ylabel("f1 weighted")
# plt.xlabel("# top features from each tool")
# plt.grid()
# plt.tight_layout()
# plt.show()
# plt.savefig(out_path+"score.eps")

# pd.DataFrame(scores, columns=['f1 weighted']).to_csv(out_path+"clf_score.csv")

Cluster 1 done for tool monocle
Cluster 1 done for tool scvi
Cluster 1 done for tool scanpy
Cluster 1 done for tool seurat
Cluster 2 done for tool monocle
Cluster 2 done for tool scvi
Cluster 2 done for tool scanpy
Cluster 2 done for tool seurat
Cluster 3 done for tool monocle
Cluster 3 done for tool scvi
Cluster 3 done for tool scanpy
Cluster 3 done for tool seurat
Cluster 4 done for tool monocle
Cluster 4 done for tool scvi
Cluster 4 done for tool scanpy
Cluster 4 done for tool seurat


In [14]:
print(scores)

[{1: {'scores': [0.8826291079812207, 0.9468599033816426, 0.966183574879227, 0.9611650485436893, 0.9607843137254903, 0.9658536585365853, 0.9615384615384616, 0.9711538461538461, 0.970873786407767, 0.966183574879227, 0.966183574879227, 0.9711538461538461, 0.9611650485436893, 0.970873786407767, 0.9807692307692307, 0.9805825242718447, 0.975609756097561, 0.9560975609756097, 0.970873786407767, 0.9758454106280193, 0.9805825242718447, 0.9658536585365853, 0.9603960396039605, 0.9658536585365853, 0.9807692307692307, 0.970873786407767, 0.9658536585365853, 0.9658536585365853, 0.9807692307692307, 0.9658536585365853, 0.9658536585365853, 0.9758454106280193, 0.975609756097561, 0.9658536585365853, 0.9805825242718447, 0.970873786407767, 0.970873786407767, 0.9758454106280193, 0.970873786407767, 0.970873786407767, 0.970873786407767, 0.9658536585365853, 0.9705882352941176, 0.9658536585365853, 0.9607843137254903, 0.9556650246305419, 0.9607843137254903, 0.9607843137254903, 0.975609756097561, 0.970873786407767,

In [ ]:
scores2 = {}
for i, el in enumerate(scores):
    scores2[list(el.keys())[0]] = el[list(el.keys())[0]]
scores = scores2

In [ ]:
# -------- train on all features and on markers --------

X_all = adata[mask, :].X.toarray()
report_all, feature_importance = apply_classifier(X_all, y)
report_markers, _ = apply_classifier(adata[mask, markers].X.toarray(), y)

pd.DataFrame(report_all).transpose().to_csv(out_path+"clf_report_all.csv")
pd.DataFrame(report_markers).transpose().to_csv(out_path+"clf_report_markers.csv")

sorted_idx = (-feature_importance).argsort()
rf_features_sorted = adata.var_names[sorted_idx]
importaces_sorted = feature_importance[sorted_idx]
pd.DataFrame(
    {'genes' : rf_features_sorted, 'importaces' : importaces_sorted}
    ).to_csv(out_path+"importances.csv")

In [ ]:
# -------- select n_markers*n_clusters features with RFE and RF --------

selector = RFE(RandomForestClassifier(), n_features_to_select=n_markers*n_clusters, step=0.5)
selector.fit(X_all, y)
sorted_idx = (selector.ranking_).argsort()
rfe_features_sorted = adata.var_names[sorted_idx]
pd.DataFrame(
    {'genes' : rfe_features_sorted}
    ).to_csv(out_path+"rfe_ranking.csv")

# automatically choose the number of features
# rfe = RFECV(estimator=RandomForestClassifier())
# rfe.fit(X_all, y)
# rfe.show()

# TODO:
# - valutare intersezione
# - valutare bontà del ranking allenando con markers più in basso nella classifica?

# important_features = features_sorted[0:120]
# important_features = [f for f, i in zip(features_sorted, importaces_sorted) if i >= importaces_sorted[119]]
# intersection = set(markers).intersection(set(important_features))

#        rank di randomforest                  rank del tool
# gene1         100 *                         * 1 - (0-20)
# gene2         1 *                            
# gene3         1 *                            
# gene4         0 *                            